In [12]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import _canoeValues as cv

input_file = "Station Information.csv"
outerHull_file = "Outer Hull.csv" # Will be used for hydrostatic force and waterline calculation

script_dir = os.path.abspath('') # absolute file path in Jupyter
file_path = os.path.join(script_dir, input_file) # file path for the station information

# Get the canoe's volume and mass from the station info
canoe_info = pd.read_csv(file_path, header=None)
statVol = canoe_info[1][1:]
statMass = canoe_info[2]

# Get the outer hull coordinates from the input file
file_path = os.path.join(script_dir, outerHull_file)
out_coor = pd.read_csv(file_path, header=None)
x_out = out_coor[0]
y_out = out_coor[1]
z_out = out_coor[2]

# Convert Series to List
stat_Mass = []
for i in range(1,len(statMass),1):
    stat_Mass.append(float(statMass[i]))

stat_Vol = []
for i in range(1,len(statVol),1):
    stat_Vol.append(float(statVol[i]))

station_no = x_out.nunique() # number of stations
station = x_out.unique() # location of each station

In [13]:
# Choose number of paddlers haha
csv_file = "Station_Loads_6 Paddlers.csv"

# Put in the numbers (from Longitudinal Analysis_X Paddler) where X is the number of paddlers I believe in you
# Separate numbers with commas so they have a sense of personal identity
paddler_positions = [500, 1867, 2595, 3156, 3883, 5250] #1161, 4563 (old)

csv_file_path = os.path.join(script_dir, csv_file)
csv_info = pd.read_csv(csv_file_path, header=None)

# Inputs ------------------------------------
F = cv.pad_male
phi_c = cv.phi_c
fc = cv.compressive_strength

bw = np.mean([float(i) for i in canoe_info[4][1:]]) # Distance between stations

lamda = cv.lamda
hf = cv.concrete_thickness
layer1 = cv.hasLayer1 # if layer exists, it equals to 1. Otherwise, zero.
layer2 = cv.hasLayer2 # if layer exists, it equals to 1. Otherwise, zero.
d_mesh_1 = cv.d_mesh_1 # distance b/w the first layer of reinforcement to the surface [mm]
d_mesh_2 = cv.d_mesh_2 # distance b/w the second layer of reinforcement to the surface [mm]
# --------------------------------------------

# Get the depth from the csv for shear and moment
#shear_file = "Shear_and_Moment_6 Paddlers.csv"
#shear_file_path = os.path.join(script_dir, csv_file)
#shear_info = pd.read_csv(csv_file_path, header=None)
#depth = shear_info[4][1]

"""
h = depth

if layer2 == 0:
    d = depth - hf + d_mesh_1
else:
    d = depth - hf + (d_mesh_1+d_mesh_2)/2

dv = max(0.9*d, 0.72*h)

"""
if layer2 == 0:
    d = hf - d_mesh_1
else:
    d = hf - (d_mesh_1+d_mesh_2)/2

dv = d

Vr = 0.25 * phi_c * fc * bw * dv * lamda
#print("The value of Vr is " + str(Vr))

#print("The values for Vf are: ")

Vf = 0

maxStation = 0
maxVf = 0
maxVr = 0


# Order of file values may be different so this might need to change
# Fsw1, Fsw2 = Self Weight
# FB1, FB2 = Buoyancy Force

# Track mm distance of station for paddler positions
mm_traveled = 0
paddler_counter = 0

# Store Vf and paddlers? for csv file
Vf_array = []
Paddler_array = []

for i in range(1,len(csv_info[0])-1,1): # <?>

    # Take in values from CSV
    FB1 = float(csv_info[1][i]) / 3
    FB2 = float(csv_info[1][i]) / 6
    Fsw1 = float(csv_info[2][i]) / 3
    Fsw2 = float(csv_info[2][i]) / 6

    # Check if a paddler is in this station and calculate accordingly
    stationHasPaddler = False
    if(paddler_counter < len(paddler_positions)):
        delta_mm_traveled = mm_traveled + float(canoe_info[4][i])
        if(mm_traveled <= paddler_positions[paddler_counter] and delta_mm_traveled > paddler_positions[paddler_counter]):
            stationHasPaddler = True
            paddler_counter += 1
        mm_traveled = delta_mm_traveled
    
    RF1 = (-1*(Fsw1+F)+FB1) if stationHasPaddler else (-1*(Fsw1)+FB1)
    RF2 = FB2 - Fsw2

    # print("RF1: " + str(RF1) + "\nRF2: " + str(RF2))

    # Kielen's gut tells her this is right
    if RF1 > 0:
        Vf = RF1 - RF2
    else:
        Vf = abs(RF1) + abs(RF2)

    # Throw values in the CSV Dungeon D:
    Vf_array.append(Vf)
    Paddler_array.append(stationHasPaddler)

    # print("\nStation " + str(i))
    # print("Vr: " + str(Vr))
    
    # if (Vr > Vf):
    #     print("Vf: " + str(Vf) + " (Vf less than Vr)")
    # elif (Vr == Vf):
    #     print("Vf: " + str(Vf) + " (Vf equal to Vr)")
    # else:
    #     print("Vf: " + str(Vf) + " (Vf greater than Vr)")

    if(Vf > maxVf):
        maxStation = i
        maxVf = Vf
        maxVr = Vr

print("Station " + str(maxStation) + " has the maximum transverse load.\nVf = " + str(maxVf) + " N\nVr = " + str(maxVr) + " N")



Station 1
Vr: 1168.6778738839287
Vf: 1.6719142752697367 (Vf less than Vr)

Station 2
Vr: 1168.6778738839287
Vf: 4.437734206937226 (Vf less than Vr)

Station 3
Vr: 1168.6778738839287
Vf: 7.621085472398271 (Vf less than Vr)

Station 4
Vr: 1168.6778738839287
Vf: 10.346981086795946 (Vf less than Vr)

Station 5
Vr: 1168.6778738839287
Vf: 68.90209651429109 (Vf less than Vr)

Station 6
Vr: 1168.6778738839287
Vf: 14.867428243484223 (Vf less than Vr)

Station 7
Vr: 1168.6778738839287
Vf: 16.788084072021377 (Vf less than Vr)

Station 8
Vr: 1168.6778738839287
Vf: 18.528200439208494 (Vf less than Vr)

Station 9
Vr: 1168.6778738839287
Vf: 20.103442107994788 (Vf less than Vr)

Station 10
Vr: 1168.6778738839287
Vf: 21.52429075698721 (Vf less than Vr)

Station 11
Vr: 1168.6778738839287
Vf: 22.79939984786421 (Vf less than Vr)

Station 12
Vr: 1168.6778738839287
Vf: 23.936847556315957 (Vf less than Vr)

Station 13
Vr: 1168.6778738839287
Vf: 24.94455070260443 (Vf less than Vr)

Station 14
Vr: 1168.677873

In [14]:
# Make a CSV file for Kielen
df1 = pd.DataFrame(data={'Station No.': range(1, len(Vf_array) + 1, 1), 'Vf': Vf_array, 'Has Paddler?': Paddler_array})

out_file = 'Transverse_Load_6 Paddlers.csv'
script_dir = os.path.abspath('')
out_file_path = os.path.join(script_dir, out_file)
df1.to_csv(out_file_path, index=False)
print("Complete!")

Complete!
